In [1]:
import numpy as np
import nibabel as nib 
import os 
import pandas as pd
from utils.utils import * 
from utils.flips import get_prefered_xy
import math

In [2]:
rois_long = {'V1v': 1, 'V1d': 2, 'V2v': 3, 'V2d': 4, 'V3v': 5, 'V3d': 6, 'hV4': 7, 'VO-1': 8,
 'VO-2': 9, 'PHC-1': 10, 'PHC-2': 11, 'LO-1': 12, 'LO-2': 13, 'TO-1': 14, 'TO-2': 15
}
sior_long = {v: k for k,v in rois_long.items()}
to_reduce = {'V1v': 'V1', 'V1d': 'V1', 'V2v': 'V2', 'V2d': 'V2', 'V3v': 'V3', 'V3d': 'V3'}
sior = {v: k for k,v in rois.items()}

In [3]:
models_subs = {}
models = ['oself']
columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang", "roi", "fit_with"]
model_study = 'oself'
mode = "train"
rotated = True

for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_rotated.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')
        models_subs[sub][m] = pd.DataFrame(np.load(m_file, allow_pickle=True), columns=columns)
        models_subs[sub][m][columns[:-2]] = models_subs[sub][m][columns[:-2]].astype(np.float32)
     #   x_prefered, y_prefered = get_prefered_xy(models_subs[sub][m]["x0"], models_subs[sub][m]["y0"], sub)  I cant do that because
     #   models_subs[sub][m]['x0_prefered'] = x_prefered  the mds have different dimension -> so problem 
      #  models_subs[sub][m]['y0_prefered'] = y_prefered
    sub_m_study = models_subs[sub][model_study]

hier_index = pd.MultiIndex.from_arrays(
    [
        np.tile(list(rois.keys()), len(models_subs)),
        np.repeat(list(models_subs.keys()), len(rois.keys())),
    ],
    names =["roi", "sub"]
)
models_subs["subj01"]["oself"]["test_var_explained"]

ValueError: shapes (2,9411) and (10000,2) not aligned: 9411 (dim 1) != 10000 (dim 0)

In [27]:
models_subs

{'subj01': {'oself':              x0        y0     sigma      slope  intercept  test_var_explained  \
  0      0.371848 -0.624069  6.235634  30.357092 -30.137138            0.000676   
  1      0.418255 -0.594880  6.208899  25.832947 -25.632463           -0.000803   
  2      0.502787 -0.546511  4.808846  12.609944 -12.428826           -0.002064   
  3      0.349385 -0.618350  5.959802  28.299004 -28.080090           -0.001289   
  4      0.331797 -0.591528  6.096788  30.145597 -29.928112           -0.002998   
  ...         ...       ...       ...        ...        ...                 ...   
  33792  0.141309 -1.050000  5.095603  48.103619 -47.682968            0.010695   
  33793 -0.109746 -1.050000  3.967579  16.174715 -15.943175            0.005295   
  33794 -0.283447  0.284247  4.676534  14.986609 -14.808397            0.008918   
  33795 -0.195607 -0.807038  4.128424  15.912541 -15.684640            0.006280   
  33796 -0.314990 -0.148864  6.484445  29.390841 -29.198298         

In [24]:
hemis = ['lh', 'rh']
fields = columns
if rotated:
    rotation = "rotated"
if not rotated: 
    rotation = "notrotated"


for model in models:
    for sub, sessions in subjects_sessions.values():
        m = models_subs[sub][model]
        print(f'------ EXPORTING MODEL FOR {sub}')
        for f in fields:
            print(f'\t{f}')
            s = -1
            for hemi in hemis:
                print(f'\t\t{hemi}')
                
                if sub == 'subj06' or sub == 'subj08':
                    maskdata_long_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.{sub}.nans.testrois.mgz')
                else:
                    maskdata_long_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.{sub}.testrois.mgz')
                maskdata_long = nib.load(maskdata_long_file).get_fdata().squeeze()
              #  print(maskdata_long.shape)
        
                maskdata_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.short.reduced.{sub}.testrois.npy')
                maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)

                data_out_file = os.path.join(results_dir, sub, f'{hemi}.{model}.{f}.{rotation}.mgz')
                label_out_file = os.path.join(label_dir,'freesurfer', sub, 'label', f'{hemi}.{model}.{f}.{rotation}.mgz') # annoying but necessary for matlab output
                if os.path.exists(data_out_file):
                    pass
                data_out = np.zeros(maskdata_long.shape)
             
                wrongs = []
                for i in range(maskdata_long.shape[0]):
                    voxel = int(maskdata_long[i])
                    if voxel==0 or voxel > 15:
                        continue
                    s += 1
                    if sior_long[voxel] in to_reduce.keys():
                        roi = rois[to_reduce[sior_long[voxel]]]
                    else:
                        roi = rois[sior_long[voxel]]
                  #  roi = rois[sior[voxel]]
                    # prove that the order is maintained
                    if roi != rois[m.iloc[s].roi]:
                        print(f'{i}: {roi} != {rois[m.iloc[s].roi]}')
                        wrongs.append(i)
                    if f in ['roi', 'fit_with']:
                        data_out[i] = rois[m.iloc[s][f]]
                    elif f == 'mds_ang':
                        data_out[i] = math.degrees(m.iloc[s][f]) + 180
                    elif f == 'intercept':
                        data_out[i] = m.iloc[s][f] + abs(m[f].min()) + 1e-08
                    elif f in ['x0', 'y0']:
                        data_out[i] = m.iloc[s][f] + 1.1
                    elif f in ['var_explained', 'test_var_explained']:
                        data_out[i] = m.iloc[s][f]  + 1 # Add 1 for the variance explain, so we can still mask the non relevant value and plot the "negative" variance
                    else:
                        data_out[i] = m.iloc[s][f]  
                img = nib.Nifti1Image(np.expand_dims(data_out, axis=(1, 2)), np.eye(4))

                nib.loadsave.save(img, data_out_file)
                nib.loadsave.save(img, label_out_file)
                if f == 'mds_ang':
                    for ang in [45, 90, 135, 180, 225, 270, 305]:
                        data_out_file = os.path.join(results_dir, sub, f'{hemi}.{model}.f.{str(ang)}.{rotation}.mgz')
                        label_out_file = os.path.join(label_dir, 'freesurfer', sub, 'label', f'{hemi}.{model}.{f}.{str(ang)}.{rotation}.mgz')
                        data_out_ang = np.copy(data_out)
                        data_out_ang[data_out != 0] += ang
                        data_out_ang = data_out_ang % 360
                        img = nib.Nifti1Image(np.expand_dims(data_out_ang, axis = (1, 2)), np.eye(4))
                        nib.loadsave.save(img, data_out_file)
                        nib.loadsave.save(img, label_out_file)


print('end')
                        
            

------ EXPORTING MODEL FOR subj01
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
------ EXPORTING MODEL FOR subj02
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
------ EXPORTING MODEL FOR subj03
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
------ EXPORTING MODEL FOR subj04
	x0
		lh
		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	test_var_explained
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh
------ EXPORTING MOD